In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import os
import numpy as np
from prosit_t.models import PrositTransformerV2
from dlomix.losses import masked_spectral_distance

2023-09-19 07:44:56.786645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 07:44:57.690443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tf.config.run_functions_eagerly(True)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
sequences_data = [
    [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9],
    [1,7],
    [4,5,6,7,8,9,9],
    [9,9,9,9,9,9,9,9,9]
]

labels_data = []
for i in sequences_data:
    labels_data.append(i*6)

charge_data = [
    [0,1,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1],
    [0,1,0,0,0,0]
]
ce_data = [
    [0.215],
    [0.24],
    [0.287],
    [0.213],
    [0.215],
    [0.234]
]
sequences_ragged = tf.ragged.constant(sequences_data, dtype=tf.int32)
labels_ragged = tf.ragged.constant(labels_data, dtype=tf.float64)
charge_ragged = tf.ragged.constant(charge_data, dtype=tf.int32)
ce_ragged = tf.ragged.constant(ce_data, dtype=tf.float64)

2023-09-19 07:44:59.965140: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43640 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [5]:
def ragged_to_dense(x,y):
    return x, y

def merge_tuples(item1, item2):
    return ({
        "sequence": item1[0],
        "collision_energy": item2[1],
        "precursor_charge": item2[0],
    }, item1[-1])

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(
            (
                sequences_ragged,
                labels_ragged,
            )
        ).map(ragged_to_dense).padded_batch(2).unbatch()

/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [7]:
dataset_meta = tf.data.Dataset.from_tensor_slices(
    (
        charge_ragged,
        ce_ragged
    )).map(ragged_to_dense)

In [8]:
full = tf.data.Dataset.zip(dataset, dataset_meta).map(merge_tuples).batch(2).prefetch(tf.data.AUTOTUNE)

In [9]:
from prosit_t.models.test_model import TestModel
import keras.backend as K

In [10]:
test = TestModel(32)

In [11]:
test.compile(loss=masked_spectral_distance, optimizer='adam')

In [13]:
test.fit(full, epochs=10)

Epoch 1/10
3/3 [==============================] - 1s 17ms/step - loss: 0.8681


2023-09-19 07:45:08.617992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 2/10
3/3 [==============================] - 0s 18ms/step - loss: 0.7160
Epoch 3/10
3/3 [==============================] - 0s 16ms/step - loss: 0.5808
Epoch 4/10
3/3 [==============================] - 0s 17ms/step - loss: 0.4667
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 0.3771
Epoch 6/10
3/3 [==============================] - 0s 16ms/step - loss: 0.3106
Epoch 7/10
3/3 [==============================] - 0s 16ms/step - loss: 0.2603
Epoch 8/10
3/3 [==============================] - 0s 17ms/step - loss: 0.2200
Epoch 9/10
3/3 [==============================] - 0s 17ms/step - loss: 0.1871
Epoch 10/10
3/3 [==============================] - 0s 16ms/step - loss: 0.1614
